In [408]:
#!pip install torch
!pip install tenseal==0.3.0a6
!pip install --upgrade pip
#!pip install pandas
#!pip install scipy
!git clone https://github.com/OpenMined/TenSEAL.git

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 2.1 MB 3.9 MB/s            
  Attempting uninstall: pip
    Found existing installation: pip 21.3.1
    Uninstalling pip-21.3.1:
      Successfully uninstalled pip-21.3.1
fatal: destination path 'TenSEAL' already exists and is not an empty directory.


In [2]:
import codecs
import json
import math

In [11]:
path = "/tf/shared/henfo/workspace/rust/bench_test/target/criterion"
types = ["LWE", "VectorLWE_10"]
operations = ["Encrypt", "Decrypt", "Addition", "Real Addition", "Sum ciphertext vector", "Interger Multiplication", \
              "Real Multiplication", "Ciphertext mult", "Bootstrap_w_func", "Keyswitch", "Encrypt vector", \
              "Decrypt vector", "Dot prod", "Real dot prod"]
lwe_keys = ["512", "1024", "2048"]

def get_time_str(t, ):
    times = ["ns", "\u03BCs", "ms", "s"]
    ind = math.floor((math.log10(t))/3)
    
    d = math.floor(math.log10(t))
    d2 = math.floor((math.log10(t))%3+1)
    t = round(t/10**(d-2))
    t = t/10**(3-d2)
    t = ('%f' % t).rstrip('0').rstrip('.') if d2 == 3 else str(t)
    return t+' '+times[ind]

Rust_Bench = {}
for typ in types:
    op_dict = {}
    for op in operations:
        key_dict = {}
        for key in lwe_keys:
            file = path+'/'+typ+'/'+op+'/'+key
            try:
                f = open(file+'/new/estimates.json')
                data = json.load(f)
                mean = data['mean']['point_estimate']
                key_dict[key] = get_time_str(mean)
                f.close()
            except:
                continue
        op_dict[op] = key_dict
    Rust_Bench[typ] = op_dict
    
for x, val in Rust_Bench['LWE'].items():
    try:
        times = [val[k] for k in lwe_keys]
        print(x, times)
    except:
        continue
        
print("\n")

for x, val in Rust_Bench['VectorLWE_10'].items():
    try:
        times = [val[k] for k in lwe_keys]
        print(x, times)
    except:
        continue

Encrypt ['25.1 μs', '48.1 μs', '95.1 μs']
Decrypt ['1.06 μs', '2.09 μs', '4.24 μs']
Addition ['1.05 μs', '2.01 μs', '3.97 μs']
Real Addition ['128 ns', '250 ns', '476 ns']
Interger Multiplication ['1.09 μs', '1.96 μs', '3.85 μs']
Real Multiplication ['1.21 μs', '2.04 μs', '3.94 μs']
Ciphertext mult ['111 ms', '448 ms', '1.8 s']
Bootstrap_w_func ['53.2 ms', '213 ms', '871 ms']
Keyswitch ['3.97 ms', '16.4 ms', '65.3 ms']


Addition ['10.3 μs', '20.1 μs', '40.2 μs']
Real Addition ['1.31 μs', '2.42 μs', '4.72 μs']
Sum ciphertext vector ['10.8 μs', '21.2 μs', '41.7 μs']
Interger Multiplication ['10.6 μs', '20.2 μs', '39.3 μs']
Real Multiplication ['13.0 μs', '23.5 μs', '44.9 μs']
Ciphertext mult ['1.11 s', '4.48 s', '18.1 s']
Bootstrap_w_func ['537 ms', '2.13 s', '8.59 s']
Keyswitch ['49.8 ms', '201 ms', '838 ms']
Encrypt vector ['237 μs', '460 μs', '941 μs']
Decrypt vector ['9.04 μs', '17.6 μs', '36.7 μs']
Dot prod ['1.11 s', '4.47 s', '18.1 s']
Real dot prod ['22.6 μs', '43.0 μs', '83.3 μ

In [411]:
import math
import numpy as np
import tenseal as ts

def add_single_ctxt(x, y):
    return x+y

def add_single_ptxt(x, p):
    return x+p

def mult_int(x, p):
    return x*p

def mult_real(x, p):
    return x*p

def mult_ctxt(x, y):
    return x*y

def sigmoid(x):
    return x.polyval([0.5, 0.197, 0, -0.004])

def add_ctxt_vector(x, y):
    return x+y

def add_ptxt_vector(x, p):
    return x+p

def mult_int_vector(x, p):
    return x*p

def mult_real_vector(x, p):
    return x*p

def mult_ctxt_vector(x, y):
    return x*y

def sigmoid_vector(x):
    return x.polyval([0.5, 0.197, 0, -0.004])

def dot_ptxt_vector(x, p):
    return x.dot(p)

def dot_ctxt_vector(x, y):
    return x.dot(y)

def encrypt(ctx_eval, x):
    return ts.ckks_vector(ctx_eval, x)

def decrypt(x):
    return x.decrypt()

def sum_vector(x):
    return x.sum()

# parameters
pmd = [8192, 16384]
ckks_keys = [str(pmd[0]), str(pmd[1])]
CKKS_Scalar = {str(pmd[0]): {}, str(pmd[1]): {}}
CKKS_Vector = {str(pmd[0]): {}, str(pmd[1]): {}}

for poly_mod_degree in pmd:
    coeff_mod_bit_sizes = [40, 20, 20, 40]
    ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
    ctx_eval.global_scale = 2 ** 20
    ctx_eval.generate_galois_keys()

    x = [0 for _ in range(10)]
    enc_vec = ts.ckks_vector(ctx_eval, x)
    enc_scalar = ts.ckks_vector(ctx_eval, [0])
    
    print("\nEncrypt")
    CKKS_Scalar[str(poly_mod_degree)]['Encrypt'] = %timeit -o encrypt(ctx_eval, [0])
    
    print("\nDecrypt")
    CKKS_Scalar[str(poly_mod_degree)]['Decrypt'] = %timeit -o decrypt(enc_scalar)
    
    print("\nadd ctxt:")
    enc_scalar2 = ts.ckks_vector(ctx_eval, [0])
    CKKS_Scalar[str(poly_mod_degree)]['Cipher Addition'] = %timeit -o add_single_ctxt(enc_scalar, enc_scalar2)

    print("\nadd plain txt:")
    CKKS_Scalar[str(poly_mod_degree)]['Real Addition'] = %timeit -o add_single_ptxt(enc_scalar, 1.)

    print("\nmult integer:")
    CKKS_Scalar[str(poly_mod_degree)]['Multiply Integer'] = %timeit -o mult_int(enc_scalar, 1)

    print("\nmult real:")
    CKKS_Scalar[str(poly_mod_degree)]['Multiply Real'] = %timeit -o mult_real(enc_scalar, 1.00)

    print("\nmult ctxt:")
    enc_scalar2 = ts.ckks_vector(ctx_eval, [0])
    CKKS_Scalar[str(poly_mod_degree)]['Multiply Cipher'] = %timeit -o mult_ctxt(enc_scalar, enc_scalar2)

    print("\nevaluate sigmoid:")
    CKKS_Scalar[str(poly_mod_degree)]['f'] =  %timeit -o sigmoid(enc_scalar)


    print("\n---- VECTOR ---- ")
    
    print("Encrypt")
    CKKS_Vector[str(poly_mod_degree)]['Encrypt'] = %timeit -o encrypt(ctx_eval, x)
    
    print("Decrypt")
    enc_vec = ts.ckks_vector(ctx_eval, x)
    CKKS_Vector[str(poly_mod_degree)]['Decrypt'] = %timeit -o decrypt(enc_vec)
    
    print("add ctxt:")
    enc_vec = ts.ckks_vector(ctx_eval, x)
    enc_vec2 = ts.ckks_vector(ctx_eval, x)
    CKKS_Vector[str(poly_mod_degree)]['Cipher Addition'] = %timeit -o add_ctxt_vector(enc_vec, enc_vec2)
    
    print("add ptxt:")
    enc_vec = ts.ckks_vector(ctx_eval, x)
    CKKS_Vector[str(poly_mod_degree)]['Real Addition'] = %timeit -o add_ptxt_vector(enc_vec, 1.)

    print("\nmult integer:")
    enc_vec = ts.ckks_vector(ctx_eval, x)
    CKKS_Vector[str(poly_mod_degree)]['Multiply Integer'] = %timeit -o mult_int_vector(enc_vec, x)

    print("\nmult real:")
    y = [1. for _ in range(10)]
    enc_vec = ts.ckks_vector(ctx_eval, x)
    CKKS_Vector[str(poly_mod_degree)]['Multiply Real'] = %timeit -o mult_real_vector(enc_vec, y)

    print("\nmult ctxt:")
    enc_vec = ts.ckks_vector(ctx_eval, x)
    enc_vec2 = ts.ckks_vector(ctx_eval, x)
    CKKS_Vector[str(poly_mod_degree)]['Multiply Cipher'] = %timeit -o mult_ctxt_vector(enc_vec, enc_vec2)

    print("\nevaluate sigmoid:")
    enc_vec = ts.ckks_vector(ctx_eval, x)
    CKKS_Vector[str(poly_mod_degree)]['f'] = %timeit -o sigmoid(enc_vec)

    print("\ndot product plain txt:")
    enc_vec = ts.ckks_vector(ctx_eval, x)
    CKKS_Vector[str(poly_mod_degree)]['Real dot'] = %timeit -o dot_ptxt_vector(enc_vec, x)

    print("\ndot product ctxt:")
    enc_vec = ts.ckks_vector(ctx_eval, x)
    enc_vec2 = ts.ckks_vector(ctx_eval, x)
    CKKS_Vector[str(poly_mod_degree)]['Cipher dot'] = %timeit -o dot_ctxt_vector(enc_vec, enc_vec2)
    
    print("\ndot product plain txt:")
    enc_vec = ts.ckks_vector(ctx_eval, x)
    CKKS_Vector[str(poly_mod_degree)]['Sum'] = %timeit -o sum_vector(enc_vec)


Encrypt


KeyboardInterrupt: 

In [ ]:
for key, x in CKKS['Vector'].items():
    print(key)

print('\n\n')
    
for key, x in CKKS['Scalar'].items():
    print(key)
    
print('\n\n')

for key, x in Rust_Bench['VectorLWE_10'].items():
    print(key)

In [405]:
import numpy as np
plain = {'Scalar': {}, 'Vector': {}}

def add_single(x):
    return x+x

def mult_int(x):
    return x*0

def mult_real(x):
    return x*0.00

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def add_vector(x):
    return x+x

def add_vector2(x):
    y = x+x
    return y

def mult_vector(x, p):
    return x*p

def dot_vector(x):
    return np.dot(x, x)

def sum_vector(x):
    return sum(x)

vec = np.array([0 for i in range(24)]) 
print("Add scalar:")
plain['Scalar']['Addition'] = %timeit -o add_single(0)

print("\nMult int:")
plain['Scalar']['Int Multiplication'] = %timeit -o mult_int(0)

print("\nMult real:")
plain['Scalar']['Real Multiplication'] = %timeit -o mult_real(0)

print("\nEvaluate sigmoid:")
plain['Scalar']['f'] = %timeit -o sigmoid(0.)

print("\n---- VECTOR ----")
print("Add vector:")
plain['Vector']['Addition'] = %timeit -o add_vector(vec)

print("\nMult int:")
plain['Vector']['Int Multiplication'] = %timeit -o mult_int(vec)

print("\nMult real:")
plain['Vector']['Real Multiplication'] = %timeit -o  mult_real(vec)

print("\nMult vector:")
real_vec = np.array([0.00 for i in range(24)])
plain['Vector']['Addition'] = %timeit -o mult_vector(vec, real_vec)

print("\nEvaluate sigmoid:")
plain['Vector']['f'] = %timeit -o sigmoid(vec)

print("\nDot product:")
plain['Vector']['Dotn'] = %timeit -o dot_vector(vec)

print("\nSum product:")
plain['Vector']['Sum'] = %timeit -o sum_vector(vec)

Add scalar:
76.9 ns ± 0.545 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)

Mult int:
72.9 ns ± 4.99 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)

Mult real:
86.2 ns ± 1.01 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)

Evaluate sigmoid:
1.19 µs ± 14.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

---- VECTOR ----
Add vector:
478 ns ± 2.54 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

Mult int:
709 ns ± 7.21 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

Mult real:
1.2 µs ± 27.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

Mult vector:
962 ns ± 14.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

Evaluate sigmoid:
3.77 µs ± 64.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Dot product:
1.31 µs ± 18.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

Sum product:
2.83 µs ± 165 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

In [4]:
import csv
csv_file = "bench.csv"
csv_columns = ['plain', ckks_keys[0], ckks_keys[1], lwe_keys[0], lwe_keys[1], lwe_keys[2]]

operations = ["Encrypt", "Decrypt", "Cipher Addition", "Real Addition", "Multiply Integer", "Multiply Real", \
              "Multiply Cipher", "f", "Real dot", "Cipher dot", "Sum"]

print(csv_columns)


# Scalar
print("\nPlain")
for key, x in plain['Scalar'].items():
    print(key, get_time_str(x.average*10**9))

print("\nCKKS")
for key, _ in CKKS_Scalar.items():
    print('\n',key)
    for typ, x in CKKS_Scalar[key].items():
        print(typ, ' ',get_time_str(x.average*10**9))

print("\nLWE")
for x, val in Rust_Bench['LWE'].items():
    try:
            print(x, '& ', val[lwe_keys[0]], '& ', val[lwe_keys[1]], '& ', val[lwe_keys[2]])
        #times = [x +' &  '+ val[k] for k in lwe_keys]        
    except:
        continue


# Vector
print("\nPlain")
for key, x in plain['Vector'].items():
    print(key, get_time_str(x.average*10**9))

print("\nCKKS")
for key, _ in CKKS_Vector.items():
    print('\n',key)
    for typ, x in CKKS_Vector[key].items():
        print(typ, ' ', get_time_str(x.average*10**9))
        try:
                print(x, '& ', val[ckks_keys[0]], '& ', val[ckks_keys[1]])       
        except:
            continue

print("\nVectorLWE")
for x, val in Rust_Bench['VectorLWE_10'].items():
    try:
            print(x, '& ', val[lwe_keys[0]], '& ', val[lwe_keys[1]], '& ', val[lwe_keys[2]])       
    except:
        continue

NameError: name 'ckks_keys' is not defined

In [12]:
import csv
csv_file = "bench.csv"
csv_columns = ['plain', lwe_keys[0], lwe_keys[1], lwe_keys[2]]

operations = ["Encrypt", "Decrypt", "Cipher Addition", "Real Addition", "Multiply Integer", "Multiply Real", \
              "Multiply Cipher", "f", "Real dot", "Cipher dot", "Sum"]

print(csv_columns)

# Scalar

print("\nLWE")
for x, val in Rust_Bench['LWE'].items():
    try:
            print(x, '& ', val[lwe_keys[0]], '& ', val[lwe_keys[1]], '& ', val[lwe_keys[2]])
        #times = [x +' &  '+ val[k] for k in lwe_keys]        
    except:
        continue


# Vector
print("\nVectorLWE")
for x, val in Rust_Bench['VectorLWE_10'].items():
    try:
            print(x, '& ', val[lwe_keys[0]], '& ', val[lwe_keys[1]], '& ', val[lwe_keys[2]])       
    except:
        continue

['plain', '512', '1024', '2048']

LWE
Encrypt &  25.1 μs &  48.1 μs &  95.1 μs
Decrypt &  1.06 μs &  2.09 μs &  4.24 μs
Addition &  1.05 μs &  2.01 μs &  3.97 μs
Real Addition &  128 ns &  250 ns &  476 ns
Interger Multiplication &  1.09 μs &  1.96 μs &  3.85 μs
Real Multiplication &  1.21 μs &  2.04 μs &  3.94 μs
Ciphertext mult &  111 ms &  448 ms &  1.8 s
Bootstrap_w_func &  53.2 ms &  213 ms &  871 ms
Keyswitch &  3.97 ms &  16.4 ms &  65.3 ms

VectorLWE
Addition &  10.3 μs &  20.1 μs &  40.2 μs
Real Addition &  1.31 μs &  2.42 μs &  4.72 μs
Sum ciphertext vector &  10.8 μs &  21.2 μs &  41.7 μs
Interger Multiplication &  10.6 μs &  20.2 μs &  39.3 μs
Real Multiplication &  13.0 μs &  23.5 μs &  44.9 μs
Ciphertext mult &  1.11 s &  4.48 s &  18.1 s
Bootstrap_w_func &  537 ms &  2.13 s &  8.59 s
Keyswitch &  49.8 ms &  201 ms &  838 ms
Encrypt vector &  237 μs &  460 μs &  941 μs
Decrypt vector &  9.04 μs &  17.6 μs &  36.7 μs
Dot prod &  1.11 s &  4.47 s &  18.1 s
Real dot prod &  

In [7]:
import csv
csv_file = "bench.csv"
csv_columns = ['plain', lwe_keys[0], lwe_keys[1], lwe_keys[2]]

operations = ["Encrypt", "Decrypt", "Cipher Addition", "Real Addition", "Multiply Integer", "Multiply Real", \
              "Multiply Cipher", "f", "Real dot", "Cipher dot", "Sum"]

print(csv_columns)

# Scalar

print("\nLWE")
for x, val in Rust_Bench['LWE'].items():
    try:
            print(x, '& ', val[lwe_keys[0]], '& ', val[lwe_keys[1]], '& ', val[lwe_keys[2]])
        #times = [x +' &  '+ val[k] for k in lwe_keys]        
    except:
        continue


# Vector
print("\nVectorLWE")
for x, val in Rust_Bench['VectorLWE_10'].items():
    try:
            print(x, '& ', val[lwe_keys[0]], '& ', val[lwe_keys[1]], '& ', val[lwe_keys[2]])       
    except:
        continue

['plain', '512', '1024', '2048']

LWE
Encrypt &  22.5 μs &  43.8 μs &  85.6 μs
Decrypt &  217 ns &  411 ns &  836 ns
Addition &  355 ns &  610 ns &  1.21 μs
Real Addition &  94.5 ns &  176 ns &  471 ns
Interger Multiplication &  191 ns &  246 ns &  358 ns
Real Multiplication &  427 ns &  721 ns &  1.14 μs
Ciphertext mult &  49.7 ms &  195 ms &  804 ms
Bootstrap_w_func &  24.2 ms &  96.2 ms &  393 ms
Keyswitch &  851 μs &  4.73 ms &  20.7 ms

VectorLWE
Addition &  3.13 μs &  6.22 μs &  12.1 μs
Real Addition &  1.27 μs &  2.31 μs &  4.66 μs
Sum ciphertext vector &  1.59 μs &  4.41 μs &  6.0 μs
Interger Multiplication &  3.69 μs &  6.75 μs &  12.8 μs
Real Multiplication &  5.61 μs &  9.66 μs &  17.8 μs
Ciphertext mult &  491 ms &  1.97 s &  8.1 s
Bootstrap_w_func &  255 ms &  970 ms &  3.94 s
Keyswitch &  8.77 ms &  48.3 ms &  206 ms
Encrypt vector &  215 μs &  424 μs &  860 μs
Decrypt vector &  2.36 μs &  4.42 μs &  8.45 μs
Dot prod &  493 ms &  1.95 s &  8.01 s
Real dot prod &  6.14 μs 